# Charting the Welsh Election
The following code is an exploration of the 2021 Welsh Senedd Election results. The main point of consideration is the difference in voting patterns and resulting seats when comparing First Past the Post (FPTP) and the more proportionally representative D'Hondt voting systems. The Welsh Senedd uses the FPTP system to allocate 40 out of a total of 60 seats for the individual constituencies.  The remaining 20 seats are then allocated with the D'Hondt method using votes from the six regions.   

In [12]:
from bs4 import BeautifulSoup as BS
import pandas as PD
import re
import requests
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches

''' WEB SCRAPING '''

# Importing raw HTML from internet
webpage = requests.get("http://www.electionpolling.co.uk/results/2021w")

# Creating soup from raw HTML object
soup = BS (webpage.text, 'html.parser')

# Extracting relevant data from soup and creating Pandas data frames
for count, table in enumerate (soup.select('.RESPTable')+soup.select('.RESVSTable')):
    table_headers = []
    table_data = []
    for th in table.find_all('th'):
        table_headers.append(th.string)
    for tr in table.find_all('tr'):
        row_data = []
        for td in tr.find_all('td', string=True):
            row_data.append(td.string)
        if row_data:
            table_data.append (row_data)
    if count == 0:
        parliament_df = PD.DataFrame (table_data, columns = table_headers)
    elif count == 1:
        constituency = PD.DataFrame (table_data, columns = table_headers)
    else:
        regional = PD.DataFrame (table_data, columns = table_headers)

constituency = constituency.iloc[:10,:3]
regional = regional.iloc[:10,:3]

In [18]:
regional

,Party,Seats,Votes
0,Labour,3,"401,770"
1,Conservative,8,"278,560"
2,Plaid Cymru,8,"230,161"
3,Green,0,"48,714"
4,Liberal Democrat,1,"48,217"
5,Abolish the Welsh Assembly,0,"41,399"
6,UKIP,0,"17,341"
7,Reform UK,0,"11,730"
8,Minor,0,"12,796"
9,Others,0,"20,207"


### Table 1 - Initial Final Regional Result Import
Shows the final result from the regional voting with the total seat and vote tally for each party. All of the data is currently in the string format and requires conversion and further calculation to prepare for plotting.

In [7]:
''' DATA CLEANING & PREPARATION '''
# Party colour dictionary with RGB hex codes taken directly from logos. Labour red toned down to reduce visual intensity.
colours = {'Labour':'#d72727ff','Conservative':'#00b0efff','Plaid Cymru':'#1b5f54ff','Liberal Democrat':'#fbc514ff', 'Abolish the Welsh Assembly':'#810000ff','Green':'#67b437ff','Others':'#d3d3d3ff'} 
tables = [constituency, regional]
for table in tables:
    # Converting string data to integers
    table['Seats'] = table['Seats'].astype(int)
    table['Votes'] = table['Votes'].str.split(',').str.join('').astype(int)
    # Simplifying minor parties into one data point
    table.at[9,'Votes'] = table.iloc[6:,2].sum()
    table.drop([6,7,8], inplace=True)
    # Calculating Percentages
    table['Votes_%'] = table.Votes/table.Votes.sum() * 100
    table['Seats_%'] = table.Seats/table.Seats.sum() * 100
    table['Difference'] = table['Seats_%'] - table['Votes_%']
    # Re-jigging index for concatenation below
    table.index = table.Party
    table.drop ('Party', axis=1, inplace=True)
    table['Colours'] = PD.Series (colours,name='Colours')
regional

,Seats,Votes,Votes_%,Seats_%,Difference,Colours
Party,,,,,,
Labour,3,401770,36.166334,15.0,-21.166334,#d72727ff
Conservative,8,278560,25.075277,40.0,14.924723,#00b0efff
Plaid Cymru,8,230161,20.718520,40.0,19.281480,#1b5f54ff
Green,0,48714,4.385113,0.0,-4.385113,#67b437ff
Liberal Democrat,1,48217,4.340374,5.0,0.659626,#fbc514ff
Abolish the Welsh Assembly,0,41399,3.726635,0.0,-3.726635,#810000ff
Others,0,62074,5.587747,0.0,-5.587747,#d3d3d3ff


In [8]:
# Creating dataframe for horizontal stacked bar chart with parties in same order for ease of visual comparison
horizontal_chart = PD.concat ([regional['Votes_%'], constituency['Votes_%']], axis=1)
horizontal_chart.columns = ['Region_Vote_%', 'Constituent_Vote_%']
horizontal_chart['Constituent_Cum'] = np.cumsum(horizontal_chart['Constituent_Vote_%']) - horizontal_chart['Constituent_Vote_%']
horizontal_chart['Region_Cum'] = np.cumsum(horizontal_chart['Region_Vote_%']) - horizontal_chart['Region_Vote_%']
horizontal_chart

,Region_Vote_%,Constituent_Vote_%,Constituent_Cum,Region_Cum
Party,,,,
Labour,36.166334,39.852032,0.000000,0.000000
Conservative,25.075277,26.067660,39.852032,36.166334
Plaid Cymru,20.718520,20.272548,65.919693,61.241611
Green,4.385113,1.602637,86.192241,81.960131
Liberal Democrat,4.340374,4.875464,87.794878,86.345244
Abolish the Welsh Assembly,3.726635,1.632501,92.670343,90.685618
Others,5.587747,5.697157,94.302843,94.412253
